In [5]:
import xlsxwriter
import pandas as pd

from pandas import DataFrame

path = ('Waste.xlsx')

xl = pd.ExcelFile(path)

print(xl.sheet_names)

['Waste management', 'Waste&WasteWaterManagement', 'Kombination']


In [6]:
wm = xl.parse('Waste management')
wwwm = xl.parse('Waste&WasteWaterManagement')

In [7]:
print(wm.columns)

Index(['FacilityID', 'FacilityName', 'City', 'Lat', 'Long', 'MainIASectorName',
       'MainIAActivityName', 'MainActivityIndicator', 'TotalQuantity1',
       'TotalQuantity2'],
      dtype='object')


In [8]:
print(wm['FacilityName'])

0     Abfallentsorgungs- und Verwertungs- gesellscha...
1     Abfallwirtschafts- und Stadtreinigungsbetrieb ...
2     AMK Abfallentsorgungsgesellschaft des Märkisch...
3     AVA Abfallverbrennungsanlage Nordweststadt Gmb...
4                                              AVA GmbH
5               AVG Abfall-Verwertungs-Gesellschaft mbH
6                                         AVG Köln GmbH
7     AWG Abfallwirtschaftsgesellschaft m bH Wupppertal
8                        BASF AG Klärschlammverbrennung
9                   Bayer Industry Services GmbH Co.OHG
10                               Biomasse-Heizkraftwerk
11                   Braunschweigische Kohlen-Bergwerke
12            Bremerhavener Entsorgungsgesellschaft mbH
13                                     BSR MVA-RUHLEBEN
14    EGK Entsorgungsgesellschaft Krefeld GmbH & Co. KG
15                           Fortum Enertec Hameln GmbH
16    Gemeinschafts-Müll-Verbrennungs- Anlage Nieder...
17                GML-Abfallwirtschaftsgesellsch

In [9]:
wmNames = wm['FacilityName']
wwwmNames = wwwm['FacilityName']

In [10]:
for name in wmNames:
    for wName in wwwmNames:
        if (name == wName):
            print(name)

Abfallentsorgungs- und Verwertungs- gesellschaft Köln mbH
HEAG Südhessische Energie AG
MHKW Coburg
MHKW Mannheim
MHKW Pirmasens
Müllheizkraftwerk Kempten
Müllkraftwerk Schwandorf
MVA Weisweiler GmbH & Co. KG
MVR Müllverwertung Rugenberger Damm GmbH & Co. KG
RWE Power AG MHKW Karnap
Stadtwerke Düsseldorf AG


In [11]:
for i in range(0, len(wmNames)):
    name = wmNames[i]
    for j in range(0, len(wwwmNames)):
        wName = wwwmNames[j]
        if (name == wName):
            print(name, "liste1:", i, wm['Lat'][i] , " Liste2:", j,  wwwm['Lat'][j])

Abfallentsorgungs- und Verwertungs- gesellschaft Köln mbH liste1: 0 51.002891  Liste2: 36 51.0028322543057
HEAG Südhessische Energie AG liste1: 19 49.884135  Liste2: 10 49.8841046677176
MHKW Coburg liste1: 23 50.285289  Liste2: 9 50.2852883741028
MHKW Mannheim liste1: 24 49.523143  Liste2: 43 49.5213819157734
MHKW Pirmasens liste1: 25 49.230269  Liste2: 50 49.2280465579331
Müllheizkraftwerk Kempten liste1: 26 47.763901  Liste2: 34 47.7588634843541
Müllkraftwerk Schwandorf liste1: 27 49.310035  Liste2: 56 49.3100349557949
MVA Weisweiler GmbH & Co. KG liste1: 32 50.840035  Liste2: 15 50.8379940631326
MVR Müllverwertung Rugenberger Damm GmbH & Co. KG liste1: 33 53.521185  Liste2: 23 53.5211584732793
RWE Power AG MHKW Karnap liste1: 35 51.515  Liste2: 16 51.5150030748136
Stadtwerke Düsseldorf AG liste1: 37 51.224705  Liste2: 12 51.2256732427172


In [16]:
lat1 = wm['Lat']
long1 = wm['Long']
lat2 = wwwm['Lat']
long2 = wwwm['Long']

results = {}

for i in range(0, len(lat1)):
    lat11 = lat1[i]
    long11 = long1[i]
    for a in range(0, len(lat2)):
        lat22 = lat2[a]
        long22 = long2[a]
        latDelta = abs(lat11 - lat22)
        longDelta = abs(long11 - long22)
        if latDelta<0.0002 and longDelta<0.0002:
            #print("Result ", long11, long22)
            tuple = {long11,long22}
            results[i] = a

for key in results:
    print(key, " - ", results[key])

3  -  17
12  -  7
17  -  40
19  -  10
23  -  9
27  -  56
28  -  35
29  -  22
33  -  23
35  -  16
41  -  8
42  -  31


In [110]:
wmData = wm.loc[ results.keys() , ['FacilityID', 'FacilityName', 'Lat', 'Long', 'MainIASectorName', 'TotalQuantity1', 'TotalQuantity2'] ]
wwwmData = wwwm.loc[ results.values() , ['FacilityID', 'FacilityName', 'Lat', 'Long', 'MainIASectorName', 'TotalQuantity1', 'TotalQuantity2'] ]

resultAsTable = [];
index = 0
for key in results:
    wmLine = wmData.loc[key,:]
    resultAsTable.append(wmLine)
    wwwmLine = wwwmData.loc[results[key],:]
    resultAsTable.append(wwwmLine)
    
dfObj = pd.DataFrame(resultAsTable) 
writer = pd.ExcelWriter('WasteResults.xlsx',engine='xlsxwriter')
workbook=writer.book
worksheet=workbook.add_worksheet('Result')
writer.sheets['Result'] = worksheet    
dfObj.to_excel(writer, sheet_name='Result', startrow=1 , startcol=0)    
  
workbook.close()
